In [3]:
import datasets
from llama_index.core.schema import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [4]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

# Initialize the tool
guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

In [5]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.fireworks import Fireworks
from dotenv import load_dotenv

import os

load_dotenv()
fw_api = os.getenv("fw_api")
# Initialize the Hugging Face model
# llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
llm = Fireworks(
    model="accounts/fireworks/models/llama4-scout-instruct-basic", api_key=fw_api
)

# Create Alfred, our gala agent, with the guest info tool
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm,
)

# Example query Alfred might receive during the gala
response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

ValueError: Unknown model: accounts/fireworks/models/llama4-scout-instruct-basic. Please provide a valid Fireworks model name.Known models are: accounts/fireworks/models/llama-v2-7b-chat, accounts/fireworks/models/llama-v2-13b-chat, accounts/fireworks/models/llama-v2-70b-chat, accounts/fireworks/models/llama-v2-34b-code-instruct, accounts/fireworks/models/llamaguard-7b, accounts/fireworks/models/llama-v3-8b-instruct, accounts/fireworks/models/llama-v3-70b-instruct, accounts/fireworks/models/llama-v3p1-8b-instruct, accounts/fireworks/models/llama-v3p1-70b-instruct, accounts/fireworks/models/llama-v3p1-405b-instruct, accounts/fireworks/models/llama-v3p2-1b-instruct, accounts/fireworks/models/llama-v3p2-3b-instruct, accounts/fireworks/models/llama-v3p2-11b-vision-instruct, accounts/fireworks/models/llama-v3p2-90b-vision-instruct, accounts/fireworks/models/mistral-7b-instruct-4k, accounts/fireworks/models/mixtral-8x7b-instruct, accounts/fireworks/models/firefunction-v1, accounts/fireworks/

In [5]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

# Initialize the DuckDuckGo search tool
tool_spec = DuckDuckGoSearchToolSpec()

search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)
# Example usage
response = search_tool("Who's the current President of France?")
print(response.raw_output[-1]['body'])

He was the Mayor of Tulle from 2001 to 2008, and was the President of the Corrèze General Council from 2008 to 2012. He was elected as President of France on 6 May 2012, defeating the incumbent Nicolas Sarkozy. Hollande was born in Rouen, Seine-Maritime, Upper Normandy, to a middle-class family.


In [6]:
import random
from llama_index.core.tools import FunctionTool


def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20},
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"


# Initialize the tool
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

In [7]:
import random
from llama_index.core.tools import FunctionTool
from huggingface_hub import list_models


def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(
            list_models(author=author, sort="downloads", direction=-1, limit=1)
        )

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"


# Initialize the tool
hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

# Example usage
print(hub_stats_tool("facebook"))  # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 22,216,629 downloads.


In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Create Alfred with all the tools
alfred = AgentWorkflow.from_tools_or_functions(
    [search_tool, weather_info_tool, hub_stats_tool], llm=llm
)

# Example query Alfred might receive during the gala
response = await alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
<think>
Okay, the user is asking two things: what Facebook is and what their most popular model is. Let me break this down.

First, I need to explain what Facebook is. I know Facebook is a social media company, now part of Meta. But maybe I should check if there's more up-to-date information. Wait, the user might be referring to Facebook the company or the platform. Since they mentioned "their most popular model," maybe they mean a machine learning model developed by Facebook's AI team. Like maybe something from their AI research division, FAIR.

So for the first part, I can briefly describe Facebook as a social media platform and part of Meta. Then, for the most popular model, I recall that Facebook has developed models like LLaMA (Large Language Model Meta AI), which is quite popular. But I should verify this. Alternatively, maybe they have other models like RoBERTa or DETR. But LLaMA has been widely discussed recently. However, I need to confirm if that's the mo